In [ ]:
import numpy as np

# Carregar os dados
dados = np.load('/content/dados_emocoes_divididos.npz')

train_ids = dados['train_ids']
train_scores = dados['train_scores']
y_train = dados['y_train']
val_ids = dados['val_ids']
val_scores = dados['val_scores']
y_val = dados['y_val']
test_ids = dados['test_ids']
test_scores = dados['test_scores']
y_test = dados['y_test']

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4804457535911295766
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14619377664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16654236342263888349
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
import tensorflow as tf
print("GPUs disponíveis:", tf.config.list_physical_devices('GPU'))

GPUs disponíveis: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import numpy as np
import itertools
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Hiperparâmetros
unidades = [64, 128, 256]
batches = [32, 64, 128, 256]
embeddings = [16, 32]
learning_rates = [0.0025, 0.005]
kernel_sizes = [3, 4, 5]
epochs = 100

num_emotions = 8
max_len = train_ids.shape[1]

resultados = []

for lstm_units, batch_size, embedding_dim, lr in itertools.product(unidades, batches, embeddings, learning_rates):
    print(f"🔄 CNN+LSTM | Batch={batch_size} | Embedding={embedding_dim} | LSTM Units={lstm_units} | LR={lr}")

    input_ids = Input(shape=(max_len,), name='emotion_ids_input')
    x = Embedding(input_dim=num_emotions, output_dim=embedding_dim, mask_zero=True)(input_ids)

    convs = []
    for k in kernel_sizes:
        c = Conv1D(filters=64, kernel_size=k, activation='relu', padding='same')(x)
        p = MaxPooling1D(pool_size=2)(c)
        convs.append(p)

    x = Concatenate(axis=-1)(convs)
    x = LSTM(lstm_units)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_ids, outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])

    nome_modelo = f"modelo_IDS_cnnmultik_lstm{lstm_units}_batch{batch_size}_emb{embedding_dim}_lr{str(lr).replace('.', '')}.keras"

    checkpoint = ModelCheckpoint(
        filepath=nome_modelo,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=0
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(
        x=train_ids,
        y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(val_ids, y_val),
        verbose=0,
        callbacks=[early_stop, checkpoint]
    )

    y_pred_probs = model.predict(val_ids)
    y_pred = (y_pred_probs > 0.5).astype(int)

    report = classification_report(y_val, y_pred, output_dict=True, zero_division=0)
    acc = report['accuracy']
    f1_fake = report.get('1', {}).get('f1-score', 0.0)

    print(f"💾 Modelo salvo: {nome_modelo}")
    print(f"✅ Resultado: Accuracy={acc:.4f} | F1-Fake={f1_fake:.4f}")

    resultados.append({
        'lstm_units': lstm_units,
        'batch_size': batch_size,
        'embedding_dim': embedding_dim,
        'learning_rate': lr,
        'accuracy': acc,
        'f1_fake': f1_fake,
        'model_file': nome_modelo
    })

# Salvar os resultados
with open('resultados_gridsearch_CNN_MultiKernel_LSTM.pkl', 'wb') as f:
    pickle.dump(resultados, f)

# Mostrar os 5 melhores
resultados.sort(key=lambda x: x['f1_fake'], reverse=True)
print("\\n🏆 Top 5 configurações por F1-Fake:")
for r in resultados[:5]:
    print(f"📂 {r['model_file']} | F1-Fake={r['f1_fake']:.4f} | Accuracy={r['accuracy']:.4f}")

🔄 CNN+LSTM | Batch=32 | Embedding=16 | LSTM Units=64 | LR=0.0025


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_1' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_2' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_cnnmultik_lstm64_batch32_emb16_lr00025.keras
✅ Resultado: Accuracy=0.6625 | F1-Fake=0.0000
🔄 CNN+LSTM | Batch=32 | Embedding=16 | LSTM Units=64 | LR=0.005


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_3' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_4' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_5' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_cnnmultik_lstm64_batch32_emb16_lr0005.keras
✅ Resultado: Accuracy=0.5760 | F1-Fake=0.0000
🔄 CNN+LSTM | Batch=32 | Embedding=32 | LSTM Units=64 | LR=0.0025


...

399/399 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
💾 Modelo salvo: modelo_IDS_cnnmultik_lstm128_batch64_emb16_lr0005.keras
✅ Resultado: Accuracy=0.5760 | F1-Fake=0.0000
🔄 CNN+LSTM | Batch=64 | Embedding=32 | LSTM Units=128 | LR=0.0025


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_42' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_43' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_44' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_cnnmultik_lstm128_batch64_emb32_lr00025.keras
✅ Resultado: Accuracy=0.5760 | F1-Fake=0.0000
🔄 CNN+LSTM | Batch=64 | Embedding=32 | LSTM Units=128 | LR=0.005


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_45' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_46' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_47' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_cnnmultik_lstm128_batch64_emb32_lr0005.keras
✅ Resultado: Accuracy=0.6464 | F1-Fake=0.0000
\n🏆 Top 5 configurações por F1-Fake:
📂 modelo_IDS_cnnmultik_lstm64_batch32_emb16_lr00025.keras | F1-Fake=0.0000 | Accuracy=0.6625
📂 modelo_IDS_cnnmultik_lstm64_batch32_emb16_lr0005.keras | F1-Fake=0.0000 | Accuracy=0.5760
📂 modelo_IDS_cnnmultik_lstm64_batch32_emb32_lr00025.keras | F1-Fake=0.0000 | Accuracy=0.6639
📂 modelo_IDS_cnnmultik_lstm64_batch32_emb32_lr0005.keras | F1-Fake=0.0000 | Accuracy=0.6453
📂 modelo_IDS_cnnmultik_lstm64_batch64_emb16_lr00025.keras | F1-Fake=0.0000 | Accuracy=0.6543


In [ ]:
# Hiperparâmetros
unidades = [32, 64, 128, 256]
batches = [32, 64, 128, 256]
embeddings = [16, 32]
learning_rates = [0.0025, 0.005, 0.01]
epochs = 100

num_emotions = 8
max_len = train_ids.shape[1]

resultados = []


for lstm_units, batch_size, embedding_dim, lr in itertools.product(unidades, batches, embeddings, learning_rates):
    print(f"🔄 LSTM={lstm_units} | Batch={batch_size} | Embedding={embedding_dim} | LR={lr}")

    input_ids = Input(shape=(max_len,), name='emotion_ids_input')
    x = Embedding(input_dim=num_emotions, output_dim=embedding_dim, mask_zero=True)(input_ids)
    x = LSTM(lstm_units)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_ids, outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])

    nome_modelo = f"modelo_IDS_lstm{lstm_units}_batch{batch_size}_emb{embedding_dim}_lr{str(lr).replace('.', '')}.keras"

    checkpoint = ModelCheckpoint(
        filepath=nome_modelo,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=0
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(
        x=train_ids,
        y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(val_ids, y_val),
        verbose=0,
        callbacks=[early_stop, checkpoint]
    )

    y_pred_probs = model.predict(val_ids)
    y_pred = (y_pred_probs > 0.5).astype(int)

    report = classification_report(y_val, y_pred, output_dict=True, zero_division=0)
    acc = report['accuracy']
    f1_fake = report.get('1', {}).get('f1-score', 0.0)

    print(f"💾 Modelo salvo: {nome_modelo}")
    print(f"✅ Resultado: Accuracy={acc:.4f} | F1-Fake={f1_fake:.4f}")

    resultados.append({
        'lstm_units': lstm_units,
        'batch_size': batch_size,
        'embedding_dim': embedding_dim,
        'learning_rate': lr,
        'accuracy': acc,
        'f1_fake': f1_fake,
        'model_file': nome_modelo
    })

# Salvar os resultados
with open('resultados_gridsearch_IDS2.pkl', 'wb') as f:
    pickle.dump(resultados, f)

# Mostrar os 5 melhores
resultados.sort(key=lambda x: x['f1_fake'], reverse=True)
print("\n -------------------------------------------------------------------------------------------------------------- \n")
print("\\n🏆 Top 5 configurações por F1-Fake:")
for r in resultados[:5]:
    print(f"📂 {r['model_file']} | F1-Fake={r['f1_fake']:.4f} | Accuracy={r['accuracy']:.4f}")

🔄 LSTM=32 | Batch=32 | Embedding=16 | LR=0.0025
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_lstm32_batch32_emb16_lr00025.keras
✅ Resultado: Accuracy=0.6635 | F1-Fake=0.0000
🔄 LSTM=32 | Batch=32 | Embedding=16 | LR=0.005
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
💾 Modelo salvo: modelo_IDS_lstm32_batch32_emb16_lr0005.keras
✅ Resultado: Accuracy=0.6683 | F1-Fake=0.0000
🔄 LSTM=32 | Batch=32 | Embedding=16 | LR=0.01
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_lstm32_batch32_emb16_lr001.keras
✅ Resultado: Accuracy=0.6658 | F1-Fake=0.0000
🔄 LSTM=32 | Batch=32 | Embedding=32 | LR=0.0025
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
💾 Modelo salvo: modelo_IDS_lstm32_batch32_emb32_lr00025.keras
✅ Resultado: Accuracy=0.6679 | F1-Fake=0.0000
🔄 LSTM=32 | Batch=32 | Embedding=32 | LR=0.005
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
💾 Modelo salvo: modelo_IDS_lstm32_batch32_emb32_lr0005.keras
✅ Resultado: Accuracy=0.6642 | F1-Fake=0.0000
🔄 LSTM=32 | Batch=32 | 